In [5]:
import sys
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
sys.path.append("E:/university/Year 5 Spring/FYT/code/multi_modal_ser")
from utils.dataset import MMSERDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")

### Audio Classification

In [8]:
import torch
from transformers import AutoFeatureExtractor, WhisperForAudioClassification

model = WhisperForAudioClassification.from_pretrained("openai/whisper-base.en").to(device)
input_features = mmser_ds[78:79]["input_features"]

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-base.en and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.weight', 'model.projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
with torch.no_grad():
    logits = model(input_features).logits

predicted_class_ids = torch.argmax(logits, axis=1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

'LABEL_1'

### Build Model

In [10]:
model.config.num_labels = mmser_ds.df_["emotion"].nunique()
model.classifier = nn.Linear(model.projector.out_features, model.config.num_labels)
# model

### Train Model

In [11]:
train_set, val_set = torch.utils.data.random_split(mmser_ds, [800, 200])
val_set, test_set = torch.utils.data.random_split(val_set, [100, 100])

In [12]:
from transformers import TrainingArguments, Trainer

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [14]:
train_loader = DataLoader(train_set, batch_size=4)
val_loader = DataLoader(val_set, batch_size=4)
test_loader = DataLoader(test_set, batch_size=4)

In [15]:
from torch import nn
from transformers import Trainer
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(device)
        # forward pass
        outputs = model(input_features=inputs["input_features"].to(device))
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss() # weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
training_args.logging_steps = 5

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# https://huggingface.co/blog/fine-tune-whisper

In [ ]:
training_args